# Test Scan merging

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from planeslam.geometry.util import quat_to_rot_mat
from planeslam.general import plot_3D_setup, NED_to_ENU
import planeslam.io as io
from planeslam.extraction import pc_to_planes
from planeslam.scan import pc_to_scan

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

Extract scans from LiDAR point clouds

In [ ]:
# Extract scans
scans = num_scans * [None]

for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])
    scans[i].transform(quat_to_rot_mat(drone_orientations[i,:]), drone_positions[i,:])

In [ ]:
# Plot scans
fig = go.Figure()

for i, scan in enumerate(scans):
    for t in scan.plot_trace():
        fig.add_trace(t)

fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

Test Reduce

In [ ]:
# Plot 2 scans
idx = 1
scan = scans[idx]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in scan.plot_trace():
    fig.add_trace(t, row=1, col=1)

scan.reduce()

for t in scan.plot_trace():
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

Merge Scans

In [ ]:
merged = scans[0]

for s in scans[1:]:
    merged = merged.merge(s)

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
merged.reduce_inside()
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()